In [1]:
# ------------------------------------------
# Script: 4 (Model_reconstruction.ipynb)
# ------------------------------------------
# Author: Pratyay Sengupta
# ------------------------------------------

In [2]:
# Import required libraries
import glob
import os
import pandas as pd


# Set data directories
input_dir = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Input_data/'
output_dir = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Output_data/'   
model_dir = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Modeling/Models/'
figures_dir = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Figures/'

In [3]:
# --------------------------------------------------------
# Load input data
# --------------------------------------------------------

# Load gram staining information
staining_info = pd.read_csv(input_dir + 'gram_staining.csv', sep=',', index_col="organism_name")

# Load genome assembly information
genome_info = pd.read_csv(output_dir + '2_genome_details.csv', sep=',', index_col="organism_name")
genome_info = genome_info[["assembly_accession"]]  # Keep only accession info

# Standardize organism names: keep only Genus species (first two words)
genome_info.index = genome_info.index.to_series().apply(lambda x: ' '.join(x.split()[:2]))

In [4]:
# --------------------------------------------------------
# Merge datasets and save
# --------------------------------------------------------

# Merge genome info with gram staining info
genome_info = genome_info.merge(staining_info, left_index=True, right_index=True)

# Save the merged genome information for modeling
genome_info.to_csv(output_dir + 'genome_details_modeling.tsv', sep='\t')

# Save list of accession IDs separately
accessions = genome_info['assembly_accession'].to_list()
accession_path = output_dir + 'accession.list'

with open(accession_path, 'w') as f:
    for accession in accessions:
        f.write("%s\n" % accession)

In [5]:
# --------------------------------------------------------
# Download assemblies and annotate (external terminal steps)
# --------------------------------------------------------

# Terminal Commands (not in script):
# conda activate bit
# bit-dl-ncbi-assemblies -w accession.list -j 80 -f fasta
# gunzip *.gz

# Run Prokka annotation (also external):
# for file in *.fasta; do prokka --outdir ../Prokka_Annotation/"$file" --prefix "$file" --cpus 40 "$file"; done

In [6]:
# --------------------------------------------------------
# Generate Bash script to run model reconstruction using CarveMe
# --------------------------------------------------------

# Paths
carve_file_path = model_dir + 'generate_models.sh'
annotated_file_path = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Modeling/Annotated_genomes/'

# Write the carve bash script
with open(carve_file_path, "w") as f:
    f.write("#!/bin/bash\n")
    for idx, row in genome_info.iterrows():
        # Carve command with gram staining-specific settings
        stain_type = 'grampos' if row['gram_staining'] == 'positive' else 'gramneg'
        f.write(f"carve {annotated_file_path}{row['assembly_accession']}.faa -o {idx.replace(' ', '_')}.xml --solver cplex -u {stain_type}\n")
        f.write(f"echo {idx} model is done\n")

print(f"Generated bash commands written to {carve_file_path}")

Generated bash commands written to /mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Modeling/Models/generate_models.sh


In [7]:
# --------------------------------------------------------
# Run the model reconstruction (run once manually)
# --------------------------------------------------------

# Change working directory
os.chdir(model_dir)

# Make script executable and run (Uncomment when running interactively)
# !chmod +x {carve_file_path}
# !./{carve_file_path}

In [8]:
# --------------------------------------------------------
# Check generated models
# --------------------------------------------------------

# List all generated XML models
models = glob.glob(model_dir + '*.xml')
models = [os.path.basename(m).replace('.xml', '').replace('_', ' ') for m in models]

# Check which genomes did not get models
model_ng = list(set(genome_info.index) - set(models))

# Print missing genome information
missing_genome_info = genome_info.loc[model_ng]

missing_genome_info

,assembly_accession,gram_staining
organism_name,,
